In [155]:
from typing import List

class Solution:
    def stoneGameIXRec(self, stones: List[int]) -> bool:
        n = len(stones)
        score = 0

        def helper(score: int, turn: int):
            # turn % 2 == 0 => Alice (True)
            # turn % 2 == 1 => Bob (False)
            player: bool = (turn % 2) == 0
            spacer = "  " * turn
            
            # if player:
            #     print(spacer + "Alice:", "turn:", turn, "score:", score, "stones:", stones)
            # else:  
            #     print(spacer + "Bob:", "turn:", turn, "score:", score, "stones:", stones)
            
            # if the score is divisible by 3, the other player wins
            if (turn > 0) and ((score % 3) == 0):
                return player
            
            # if we get to the end, Bob wins
            if turn == n:
                # print(spacer + " " + "Bob wins")
                return False

            result = not player
            for i in range(n):
                if stones[i] > 0:
                    val = stones[i]
                    stones[i] = 0
                    if helper(score + val, turn + 1) == player:
                        result = player
                    stones[i] = val
    
            return result
        
        return helper(0, 0)
    
    # formulated this way, the problem is O(2^n) without memoization
    # about O(n^2) with memoization 
    def stoneGameIXMemo(self, stones: List[int]) -> bool:
        n = len(stones)
        memo = {}
        turn = 0

        stone_dict = {}
        stone_dict[0] = 0
        stone_dict[1] = 0
        stone_dict[2] = 0
        for i in range(n):
            stone_dict[stones[i] % 3] += 1
        #print("stone_dict:", stone_dict)

        def helper(score: int, turn: int):
            n3 = stone_dict[0]
            n1 = stone_dict[1]
            n2 = stone_dict[2]

            if (n1, n2, n3) in memo:
                return memo[(n1, n2, n3)]

            player: bool = (turn % 2) == 0
            
            # if the score is divisible by 3, the previous player 
            # can't win therefore current player wins
            if (turn > 0) and ((score % 3) == 0):
                memo[(n1, n2, n3)] = player
                return memo[(n1, n2, n3)]
            
            # if we get to the end, Bob wins
            if turn == n:
                # print(spacer + " " + "Bob wins")
                memo[(n1, n2, n3)] = False
                return memo[(n1, n2, n3)]
            
            result = not player
            for i in range(0, 3):
                if stone_dict[i] > 0:
                    stone_dict[i] -= 1
                    if helper(score + i, turn + 1) == player:
                        result = player
                        stone_dict[i] += 1
                        break

                    stone_dict[i] += 1
    
            memo[(n1, n2, n3)] = result
            return memo[(n1, n2, n3)]
        
        return helper(0, max(turn-1, 0))

    # this version is basically just pure math
    def stoneGameIX(self, stones: List[int]) -> bool:
        dict = {0: 0, 1: 0, 2: 0}
        for stone in stones:
            dict[stone % 3] += 1
        
        if dict[0] % 2 == 0:
            # if we have an even number of 0s:
            # no 1's and no 2's: Bob wins
            # 1's and no 2's: Alice can't force a win
            # 2's and no 1's: Alice can't force a win
            # 1's and 2's: Alice picks whichever type has fewest and forces a win
            return dict[1] != 0 and dict[2] != 0
        else:
            # if we have an odd number of 0s:
            # can we make 2 3's?
            return abs(dict[1] - dict[2]) >= 3


In [156]:
import sys
import time
import random

random.seed(0)
sys.setrecursionlimit(10**6)

soln = Solution()
f = soln.stoneGameIX

# test 1
stone = [2,1]
expected = True
print("result:", f(stone), "expected:", expected)

# test 2
stones = [2]
expected = False
print("result:", f(stones), "expected:", expected)

# test 3
stones = [5,1,2,4,3]
expected = False
print("result:", f(stones), "expected:", expected)

# test 4
stones = [7,10,1,9,19,17,1,9,19]
expected = True
print("result:", f(stones), "expected:", expected)

# test 5
stones = [12,16,3,15,20,20,18,6,19,3]
expected = True
print("result:", f(stones), "expected:", expected)

# timing tests
print("\ntiming tests\n")

n = 100000
iter = 1
start = time.time()
for _ in range(iter):
    stones = [random.randint(1, 1000) for _ in range(n)]
    expected = None
    print("result:", f(stones), "expected:", expected)
print("time:", time.time() - start)

result: True expected: True
result: False expected: False
result: False expected: False
result: True expected: True
result: True expected: True

timing tests

result: True expected: None
time: 0.06352114677429199
